In [64]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [65]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15882753108890017186, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17961074493166266842
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3065150693228014571
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15884314215
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12800046019626638970
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

# MLP Version

In [66]:
# splitting data into train/test
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [67]:
# Converting to 1d data. 784 is calculated by 28x28 (each mnist image size)
x_train = x_train.reshape(60000, 784).astype('float32')
x_test = x_test.reshape(10000, 784).astype('float32')
x_train /= 255
x_test /= 255

x_train.shape, x_test.shape

((60000, 784), (10000, 784))

In [68]:
x_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [69]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

y_train.shape, y_test.shape

((60000, 10), (10000, 10))

In [70]:
model = Sequential()

In [71]:
model.add(Dense(512, activation = 'relu', input_shape = (784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [72]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(x_train, y_train, batch_size = 512, epochs = 50, verbose = 1, validation_data = (x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 1s 22us/step - loss: 0.3778 - acc: 0.8897 - val_loss: 0.1406 - val_acc: 0.9566
Epoch 2/50
60000/60000 [==============================] - 1s 11us/step - loss: 0.1332 - acc: 0.9608 - val_loss: 0.0949 - val_acc: 0.9700
Epoch 3/50
60000/60000 [==============================] - 1s 10us/step - loss: 0.0896 - acc: 0.9732 - val_loss: 0.0761 - val_acc: 0.9753
Epoch 4/50
60000/60000 [==============================] - 1s 11us/step - loss: 0.0677 - acc: 0.9792 - val_loss: 0.0696 - val_acc: 0.9776
Epoch 5/50
60000/60000 [==============================] - 1s 10us/step - loss: 0.0545 - acc: 0.9833 - val_loss: 0.0636 - val_acc: 0.9786
Epoch 6/50
60000/60000 [==============================] - 1s 11us/step - loss: 0.0430 - acc: 0.9865 - val_loss: 0.0609 - val_acc: 0.9816
Epoch 7/50
60000/60000 [==============================] - 1s 11us/step - loss: 0.0367 - acc: 0.9888 - val_loss: 0.0672 - val_acc

In [73]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 0s 46us/step


[0.07866956510487697, 0.9843]

# CNN Version

In [105]:
# splitting data into train/test
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [106]:
# Converting to 1d data
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28).astype('float32')
x_train /= 255
x_test /= 255

x_train.shape, x_test.shape

((60000, 1, 28, 28), (10000, 1, 28, 28))

In [107]:
# Sanity check: All values between 0 and 1
x_train[0]

array([[[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.

In [108]:
# This should be (60000, 10) and (10000, 10) representing 10 classes in MNIST => 0-9 handwritten digits
y_train.shape, y_test.shape

((60000,), (10000,))

In [109]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

y_train.shape, y_test.shape

((60000, 10), (10000, 10))

In [110]:
model = Sequential()

In [111]:
# model.add(Conv2D(32, kernel_size=(3, 3), activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(1,28,28), data_format='channels_first'))

model.add(Conv2D(64, kernel_size=(3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.2))
 
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation = 'softmax'))

In [112]:
from keras.callbacks import ModelCheckpoint

callback = ModelCheckpoint('/kaggle/working/', monitor='val_acc', verbose=1, save_best_only=False, save_weights_only=True, mode='auto', period=10)

In [113]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
# model.fit(x_train, y_train, batch_size = 64, callbacks = callback, epochs = 10, verbose = 1)
model.fit(x_train, y_train, batch_size = 64, epochs = 20, verbose = 1)

Epoch 1/20
60000/60000 [==============================] - 7s 116us/step - loss: 0.2541 - acc: 0.9231
Epoch 2/20
60000/60000 [==============================] - 6s 104us/step - loss: 0.0962 - acc: 0.9721
Epoch 3/20
60000/60000 [==============================] - 6s 104us/step - loss: 0.0726 - acc: 0.9782
Epoch 4/20
60000/60000 [==============================] - 6s 104us/step - loss: 0.0601 - acc: 0.9818
Epoch 5/20
60000/60000 [==============================] - 6s 104us/step - loss: 0.0479 - acc: 0.9850
Epoch 6/20
60000/60000 [==============================] - 6s 104us/step - loss: 0.0428 - acc: 0.9871
Epoch 7/20
60000/60000 [==============================] - 6s 104us/step - loss: 0.0373 - acc: 0.9881
Epoch 8/20
60000/60000 [==============================] - 6s 104us/step - loss: 0.0339 - acc: 0.9889
Epoch 9/20
60000/60000 [==============================] - 6s 105us/step - loss: 0.0297 - acc: 0.9908
Epoch 10/20
60000/60000 [==============================] - 6s 104us/step - loss: 0.0272 - a

In [114]:
model.evaluate(X_test, Y_test, verbose=0)

[0.04003866923764831, 0.9907]